In [298]:
import pandas as pd
import geopandas as gpd
import os
import parcelfunks
import csv
import numpy as np
from scipy import stats

In [2]:
import glob

Checking to see if lengths match


In [157]:
pop_dir = r'C:\Users\phwh9568\OneDrive - UCB-O365\UCB\Desktop\DECENNIALSF12010.H10_2023-09-21T172526'

In [158]:
counties_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties'
blocksPath = r'C:\Users\phwh9568\Data\Census2020\tl_2020_08_all\tl_2020_08_tabblock10.shp'

In [181]:
COBlocks = gpd.read_file(blocksPath, dtype={'GEOID10':str})

In [182]:
len(COBlocks.loc[COBlocks['GEOID10'].str.startswith('08059')])

10139

In [183]:
CO_BlocksPop = pd.read_csv(os.path.join(pop_dir,'Colorado_BLocks_Pop_2010.csv'), dtype={'GEOID': str})

c:\Users\phwh9568\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3524: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [184]:
len(CO_BlocksPop.loc[CO_BlocksPop['GEOID'].str.startswith('08059')])

10139

In [185]:
len(CO_BlocksPop) == len(COBlocks)

True

In [186]:
CO_BlocksPop.rename({'GEOID':'GEOID10', 'Total':'Pop_Total'}, axis='columns',inplace=True)

In [187]:
CO_BlocksPop.columns

Index(['GEO_ID', 'GEOID10', 'NAME', 'H010001', 'H010001ERR', 'Geography',
       'Geographic Area Name', 'Pop_Total', 'Errata of Total'],
      dtype='object')

In [188]:
COBlocks.columns

Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE10', 'GEOID10',
       'NAME10', 'MTFCC10', 'UR10', 'UACE10', 'UATYPE', 'FUNCSTAT10',
       'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'geometry'],
      dtype='object')

merge:

In [189]:
COBlocks = COBlocks.merge(CO_BlocksPop,on='GEOID10')

In [190]:
len(CO_BlocksPop) == len(COBlocks)

True

drop extra columns:

In [191]:
drops = CO_BlocksPop.columns.tolist()
drops.remove('Pop_Total')
drops.remove('GEOID10')

In [192]:
COBlocks.drop(drops,axis=1, inplace=True)

reproject to albers:

In [172]:
blocksAlbers = COBlocks.to_crs(crs='ESRI:102003')
blocksAlbers['blockArea_m'] = blocksAlbers['geometry'].area

split based on counties and write into county dirs:  
Hold off on this... TURNS OUT parcelInventory.csv IS BAD

In [88]:
externalDrive = r'E:/'
pInventory = pd.read_csv(os.path.join(externalDrive,'parcelInventory.csv'), dtype={'STATE':str,'COUNTY':str})
CO_path = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
CO_pInventory = pInventory.loc[pInventory['STATE']=='08']
CO_pInventory_True = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==True]
CO_pInventory_False = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==False]
CO_pInventory_False.to_csv(os.path.join(CO_path,'missingParcelData.csv'))
fipsList = CO_pInventory_True['COUNTY'].tolist()   
parcelsPaths = [os.path.join(CO_path,fips) for fips in fipsList]

In [23]:
#not used
geoidList = []
for g in blocksAlbers.GEOID10.to_list():
    geoidList.append(g[0:5])

In [77]:
# NOT USING THIS
parcelsPaths = glob.glob(r'c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties/08*')
fipsList = [p.split('\\')[-1] for p in parcelsPaths]
len(fipsList) == len(parcelsPaths)

In [396]:
externalDrive = r'E:/'
CO_path = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
pInventory = pd.read_csv(os.path.join(CO_path,'ColoradoInventory.csv'), dtype={'STATE':str,'COUNTY':str})
CO_pInventory = pInventory.loc[pInventory['STATE']=='08']
CO_pInventory_True = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==True]
CO_pInventory_False = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==False]
CO_pInventory_False.to_csv(os.path.join(CO_path,'missingParcelData.csv'))
fipsList = CO_pInventory_True['COUNTY'].tolist()   
parcelsPaths = [os.path.join(CO_path,fips) for fips in fipsList]

In [194]:
#SPLIT BLOCKS INTO COUNTY DIRS
for fips,path in zip(fipsList,parcelsPaths):
    countyBlocks = (blocksAlbers.loc[blocksAlbers['GEOID10'].str.startswith(fips)])    
    if len(countyBlocks) > 0:
        countyBlocks.to_file(os.path.join(path,fips+'_blocks.gpkg'), layer=fips+'_blocks')
    else:
        print(os.path.join(path,fips))

#### Now splitting mobile homes into county directories

Get mhps into a gpkg

In [129]:
mhps = gpd.read_file(r'C:\Users\phwh9568\Data\ParcelAtlas\Mobile_Home_Parks\MHPgeocoded.shp')

In [130]:
mhps.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [117]:
co_mhps = mhps.loc[mhps['USER_COU_1'].str.startswith('08')]

In [118]:
mhomesPath = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\08_MHPs.gpkg'

In [119]:
co_mhps.to_file(mhomesPath, layer='08_MHPs')

In [120]:
mhomesPrepped = parcelfunks.mhomes_prepper(mhomesPath)
mhomesPrepped.to_file(mhomesPath, layer='08_MHPs_Prepped')

In [124]:
mhps = mhomesPrepped

In [279]:
externalDrive = r'E:/'
CO_path = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
pInventory = pd.read_csv(os.path.join(CO_path,'ColoradoInventory.csv'), dtype={'STATE':str,'COUNTY':str})
CO_pInventory = pInventory.loc[pInventory['STATE']=='08']
CO_pInventory_True = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==True]
CO_pInventory_False = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==False]
CO_pInventory_False.to_csv(os.path.join(CO_path,'missingParcelData.csv'))
fipsList = CO_pInventory_True['COUNTY'].tolist()   
parcelsPaths = [os.path.join(CO_path,fips) for fips in fipsList]

In [156]:
#SPLIT MOBILE HOME PARKS INTO COUNTY DIRS
for fips,path in zip(fipsList,parcelsPaths):
    countyMHPs = (mhps.loc[mhps['MH_COUNTY_FIPS']==fips])    
    if len(countyMHPs) > 0:
        countyMHPs.to_file(os.path.join(path,fips+'_mhps.gpkg'), layer=fips+'_mhps')
    else:
        print(os.path.join(path,fips))

C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08017\08017
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08023\08023
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08033\08033
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08047\08047
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08079\08079
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08109\08109
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08111\08111
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08115\08115


#### Repeat above mobile home splitting using original dataset (non-regeocoded)

In [274]:
from importlib import reload
reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [128]:
mhps_OG = gpd.read_file(r'C:\Users\phwh9568\Data\ParcelAtlas\Mobile_Home_Parks\MobileHomeParks.shp')


In [134]:
mhps_OG.to_crs(4269, inplace=True)

In [135]:
co_mhps_OG = mhps_OG.loc[mhps_OG['COUNTYFIPS'].str.startswith('08')]

In [136]:
mhomesPath_OG = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\08_MHPs_OG.gpkg'

In [137]:
co_mhps_OG.to_file(mhomesPath_OG, layer='08_MHPs_OG')

In [138]:
co_mhps_OG.columns

Index(['OBJECTID', 'MHPID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'COUNTY', 'COUNTYFIPS', 'COUNTRY',
       'LATITUDE', 'LONGITUDE', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE',
       'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'UNITS', 'SIZE',
       'REVGEOFLAG', 'geometry'],
      dtype='object')

In [139]:
mhomesPrepped_OG = parcelfunks.mhomes_prepper2(mhomesPath_OG)
mhomesPrepped_OG.to_file(mhomesPath_OG, layer='08_MHPs_Prepped_OG')

In [140]:
mhomesPrepped_OG

,MHPID,MH_NAME,MH_ADDRESS,MH_CITY,MH_STATE,MH_ZIP,MH_STATUS,MH_COUNTY_FIPS,MH_LATITUDE,MH_LONGITUDE,geometry
0,41534.0,NOT AVAILABLE,938 DENVER AVE,FORT LUPTON,CO,80621,OPEN,08123,40.088103,-104.810604,POINT (-104.81060 40.08810)
1,41535.0,NOT AVAILABLE,2485 FOLSOM ST,BOULDER,CO,80304,OPEN,08013,40.025773,-105.262849,POINT (-105.26285 40.02577)
2,41537.0,FORT COLLINS KOA,LAKESIDE RESORT LN,FORT COLLINS,CO,80524,OPEN,08069,40.615220,-105.108135,POINT (-105.10814 40.61522)
3,41518.0,NOT AVAILABLE,2406 KARA CT,PUEBLO,CO,81004,OPEN,08101,38.232781,-104.639150,POINT (-104.63915 38.23278)
4,41507.0,NOT AVAILABLE,1050 W JEFFERSON AVE,HAYDEN,CO,81639,OPEN,08107,40.494103,-107.269038,POINT (-107.26904 40.49410)
...,...,...,...,...,...,...,...,...,...,...,...
679,56505.0,NOT AVAILABLE,899 BAUER AVE,MANCOS,CO,81328,OPEN,08083,37.345413,-108.302316,POINT (-108.30232 37.34541)
680,56506.0,NOT AVAILABLE,500 W GRAND AVE,MANCOS,CO,81328,OPEN,08083,37.344749,-108.295038,POINT (-108.29504 37.34475)
681,56507.0,NOT AVAILABLE,273 E MENEFEE ST,MANCOS,CO,81328,OPEN,08083,37.348561,-108.286911,POINT (-108.28691 37.34856)
682,56508.0,NOT AVAILABLE,458 LYELL ST,MONTE VISTA,CO,81144,OPEN,08105,37.575851,-106.138208,POINT (-106.13821 37.57585)


In [141]:
#SPLIT ORIGINAL MOBILE HOME PARKS INTO COUNTY DIRS
for fips,path in zip(fipsList,parcelsPaths):
    countyMHPs = (mhomesPrepped_OG.loc[mhomesPrepped_OG['MH_COUNTY_FIPS']==fips])    
    if len(countyMHPs) > 0:
        countyMHPs.to_file(os.path.join(path,fips+'_mhps_OG.gpkg'), layer=fips+'_mhps_OG_prepped')
    else:
        print(os.path.join(path,fips))

C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08017\08017
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08023\08023
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08033\08033
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08047\08047
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08079\08079
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08109\08109
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08111\08111
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08115\08115


In [52]:
# SPLITTING BLOCKS DATA INTO COUNTY DIRECTORIES

for fips, path in zip(fipsList,parcelsPaths):
    countyBlocks = blocksAlbers.loc[blocksAlbers['GEOID10'].str.startswith(fips)]
    countyBlocks.to_file(os.path.join(path,fips+'_blocks.gpkg'),layer=fips+'_blocks')

In [72]:
adams = gpd.read_file(os.path.join(parcelsPaths[0],fipsList[0]+'_blocks.gpkg'), layer=fipsList[0]+'_blocks')

In [73]:
adams.columns

Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE10', 'GEOID10',
       'NAME10', 'MTFCC10', 'UR10', 'UACE10', 'UATYPE', 'FUNCSTAT10',
       'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'Pop_Total',
       'blockArea_m', 'geometry'],
      dtype='object')

prep mobile homes:

In [400]:
for fips, path in zip(fipsList,parcelsPaths):
    if os.path.exists(os.path.join(path,fips+'2.gpkg')):
        os.remove((os.path.join(path,fips+'2.gpkg')))

In [76]:
for fips, path in zip(fipsList, parcelsPaths):
    if os.path.exists(os.path.join(path,fips+'.gpkg')):
        

c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08001\08001.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08003\08003.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08005\08005.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08007\08007.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08013\08013.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08014\08014.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08015\08015.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08019\08019.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08021\08021.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08027\08027.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08029\08029.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08031\08031.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08035\08035.gpkg
c:/users/phwh9568/data/ParcelAtlas/CO_2022/Counties\08037\08037.gpkg
c:/users/phwh9568/data/ParcelAtlas

figuring out fips problem

In [130]:
stateFips = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\stateFips.csv', dtype={'STATEFP':str})
stateFipsList = stateFips['STATEFP'].tolist()
countyFips = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\fips-by-state.csv', dtype={'fips':str}, encoding='iso-8859-1')
countyFipsList = countyFips['fips'].tolist()

In [131]:
'08059' in countyFipsList

True

In [134]:
co_fips = countyFips.loc[countyFips['fips'].str.startswith('08')]

In [139]:
co_fips_list = co_fips['fips'].to_list()

In [140]:
for fips in co_fips_list:
    if os.path.exists(os.path.join(externalDrive))

True

ah ha! for some reason, jeff co was missing from the external drive copy of the parcel data, don't know why, but probably others. Need to make sure data on the external drive matches what's on my laptop/the original parcel data.

For now, I will modify the code below to generate an inventory of just colorado based on my local copy.

In [4]:
countiesPath = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties'

In [151]:
with open(os.path.join(countiesPath,'ColoradoInventory.csv'),'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['STATE','COUNTY','DATA_PRESENT'])
    for cfip in co_fips_list:
        if os.path.exists(os.path.join(countiesPath, cfip)) :
            writer.writerow(['08',cfip,'True'])
        else:
            writer.writerow(['08',cfip,'False'])

In [149]:
for cfip in co_fips_list:
    print(os.path.join(countiesPath,cfip))

C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08001
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08003
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08005
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08007
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08009
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08011
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08013
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08014
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08015
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08017
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08019
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08021
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08023
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08025
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08027
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08029
C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\08031
C:\Users\phwh9

In [148]:
co_fips

,fips,name,state
244,08001,Adams County,CO
245,08003,Alamosa County,CO
246,08005,Arapahoe County,CO
247,08007,Archuleta County,CO
248,08009,Baca County,CO
...,...,...,...
303,08117,Summit County,CO
304,08119,Teller County,CO
305,08121,Washington County,CO
306,08123,Weld County,CO


In [196]:
parcelsPaths

['C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08001',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08003',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08005',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08007',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08013',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08014',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08015',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08017',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08019',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08021',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08023',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08027',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08029',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08031',
 'C:\\Users\\phwh9568\\Data\\Parce

Trouble shooting union merge:

In [ ]:
unioncsv = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08001\union_csv.csv')
mhpgpkg = gpd.read_file(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08001\08001.gpkg', layer='MH_parc_blk_union')

Comparing OG vs Regeocoded results:

In [401]:
regeo = pd.read_csv(os.path.join(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022', 'Colorado_regeocoded.csv'), dtype={'STATEFP10':str,'COUNTYFP10':str,'TRACTCE10':str,'BLOCKCE10':str,'GEOID10':str,'MTFCC10':str,'UACE10':str,'GEOID10':str,'GEOID10':str, 'MH_COUNTY_FIPS':str, 'MHPID':str})
og = pd.read_csv(os.path.join(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022', 'Colorado_original2.csv'), dtype={'STATEFP10':str,'COUNTYFP10':str,'TRACTCE10':str,'BLOCKCE10':str,'GEOID10':str,'MTFCC10':str,'UACE10':str,'GEOID10':str,'GEOID10':str, 'MH_COUNTY_FIPS':str, 'MHPID':str})

In [402]:
og.equals(regeo)

False

In [403]:
len(og)

1632

In [404]:
len(regeo)

1764

In [405]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'

In [476]:
mhps_regeocoded = gpd.read_file(os.path.join(co_22_dir, '08_MHPs.gpkg'))
mhps_original = gpd.read_file(os.path.join(co_22_dir, '08_MHPs_OG.gpkg'))

In [13]:
len(mhps_regeocoded), len(mhps_original)

(684, 684)

In [126]:
mhps_regeocoded.crs == mhps_original.crs

False

Trying to figure out why these are so different.  

Will combine all MH_parc_blk_union layers into one state layer and visually compare... 

In [408]:
finalDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    if os.path.exists(os.path.join(path,fips+'.gpkg')):
        county = gpd.read_file(os.path.join(path, fips+'.gpkg'),layer='MH_parc_blk_union')
        finalDF = pd.concat([finalDF,county])
finalDF.to_file(os.path.join(co_22_dir,'Colorado_Final_regeo.gpkg'),layer='Colorado_Final_regeo')

In [409]:
finalDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    if os.path.exists(os.path.join(path,fips+'2.gpkg')):
        county = gpd.read_file(os.path.join(path, fips+'2.gpkg'),layer='MH_parc_blk_union')
        finalDF = pd.concat([finalDF,county])
finalDF.to_file(os.path.join(co_22_dir,'Colorado_Final_original.gpkg'),layer='Colorado_Final_original')

In [440]:
mhp_parcels_regeo = gpd.read_file(os.path.join(co_22_dir,'Colorado_Final_regeo.gpkg'), layer='Colorado_Final_regeo_mhp-parcels')
mhp_parcels_orig = gpd.read_file(os.path.join(co_22_dir,'Colorado_Final_original.gpkg'), layer='Colorado_Final_original_mhp-parcels')

In [442]:
# write tables to csvs
mhp_parcels_regeoDF = pd.DataFrame(mhp_parcels_regeo)
mhp_parcels_origDF = pd.DataFrame(mhp_parcels_orig)
mhp_parcels_regeoDF.to_csv(os.path.join(co_22_dir,'Colorado_MHP_Parcels_regeo.csv'))
mhp_parcels_origDF.to_csv(os.path.join(co_22_dir,'Colorado_MHP_Parcels_original.csv'))

prepping for merge in order to find differences

In [444]:
mhp_parcels_orig['VERSION'] = 'original'
mhp_parcels_regeo['VERSION'] = 'regeocoded'

In [445]:
mhp_parcels_orig['MH_Geocoded_X'] = np.nan
mhp_parcels_orig['MH_Geocoded_Y'] = np.nan

In [446]:
#just re-ordering the columns here
mhp_parcels_orig = mhp_parcels_orig[['APN', 'APN2', 'MH_Geocoded_X', 'MH_Geocoded_Y', 'MHPID', 'MH_NAME',
       'MH_ADDRESS', 'MH_CITY', 'MH_STATE', 'MH_ZIP', 'MH_STATUS',
       'MH_COUNTY_FIPS', 'MH_LATITUDE', 'MH_LONGITUDE', 'geometry', 'VERSION']]

In [447]:
mhp_parcels_regeo.columns == mhp_parcels_orig.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

merged: this df is only MHPIDs that are in both datasets. Some will be to same parcels, some will be to different parcels.

In [448]:
merged = mhp_parcels_orig.merge(mhp_parcels_regeo, on='MHPID', how='inner')

In [449]:
len(mhp_parcels_regeo), len(mhp_parcels_orig)

(442, 211)

In [450]:
len(merged)

178

In [451]:
merged['match'] = merged['APN_x'] == merged['APN_y'] #creates a boolean column

merged_diff represents all mhp/parcels that matched in both versions but to different parcels.

In [452]:
merged_diff = merged.loc[merged['match']==False]

In [455]:
merged_same = merged.loc[merged['match']==True]

In [456]:
len(merged_same)

106

In [453]:
len(merged_diff)

72

In [514]:
merged_diff.drop(['geometry_x','geometry_y'],axis=1).to_csv(os.path.join(co_22_dir,'Colorado_diffs.csv'))

finding the mhp/parcels matched in the regeocoded data but not matched in original

In [468]:
mhp_parcels_concat = pd.concat([mhp_parcels_orig, mhp_parcels_regeo])

In [469]:
len(mhp_parcels_concat)

653

In [470]:
mhp_parcels_unique = mhp_parcels_concat.drop_duplicates(subset=['MHPID'], keep=False)

In [471]:
len(mhp_parcels_unique)

300

In [465]:
mhp_parcels_unique_regeo = mhp_parcels_unique.loc[mhp_parcels_unique['VERSION']=='regeocoded']
mhp_parcels_unique_orig = mhp_parcels_unique.loc[mhp_parcels_unique['VERSION']=='original']

In [466]:
len(mhp_parcels_unique_regeo), len(mhp_parcels_unique_orig)

(264, 36)

In [430]:
mhp_parcels_unique_regeo.to_file(os.path.join(co_22_dir, 'Colorado_Final_regeo.gpkg'), layer='mhp_parcels_unique_regeo')
mhp_parcels_unique_orig.to_file(os.path.join(co_22_dir, 'Colorado_Final_original.gpkg'), layer='mhp_parcels_unique_orig')

Find non-matched:

In [488]:
len(mhp_parcels_concat), len(mhps_original)

(653, 684)

In [494]:
mhps = mhps_original.merge(mhp_parcels_concat, on='MHPID', how='outer')

In [495]:
len(mhps)

862

In [503]:
mhps_noMatch = mhps.loc[mhps['VERSION'].isna()]

In [504]:
len(mhps_noMatch)

209

In [506]:
MHPIDS = mhps_original['MHPID'].unique().tolist()
MHPIDS_matched = mhp_parcels_concat['MHPID'].unique().tolist()

In [507]:
yes = 0
no = 0
for mhpid in MHPIDS:
    if mhpid in MHPIDS_matched:
        yes+=1
    else:
        no+=1

print('Yes:',yes)
print('No:',no)

Yes: 475
No: 209


In [508]:
475+209

684

In [509]:
mhps_noMatch.to_csv(os.path.join(co_22_dir,'mhps_nonmatched.csv'))

filtering out complex parcel geometries (which would typically be street grids or roads)

In [164]:
mhp_parcels_unique_regeo.geometry

1      POLYGON ((-104.43109 39.75824, -104.43149 39.7...
2      POLYGON ((-104.42858 39.75536, -104.42858 39.7...
3      POLYGON ((-104.78241 39.98664, -104.78239 39.9...
6      POLYGON ((-105.01078 39.86897, -105.01077 39.8...
7      POLYGON ((-105.01564 39.87578, -105.01563 39.8...
                             ...                        
455    POLYGON ((-104.90152 40.33344, -104.90150 40.3...
457    POLYGON ((-104.85947 40.33010, -104.85947 40.3...
460    POLYGON ((-104.80975 40.08772, -104.80998 40.0...
461    POLYGON ((-105.02906 40.16105, -105.03220 40.1...
465    POLYGON ((-104.92932 40.08744, -104.92930 40.0...
Name: geometry, Length: 258, dtype: geometry

In [182]:
poly = mhp_parcels_unique_regeo.iloc[59].geometry

In [202]:
len(poly.exterior.xy[0])

1665

In [203]:
poly2 = mhp_parcels_unique_regeo.iloc[0].geometry

In [279]:
def geomLen(geom):
    return len(geom.exterior.xy[0])

In [309]:
def geomZscore(geom):
    return stats.zscore(len(geom.exterior.xy[0]))

In [360]:
testdf = mhp_parcels_unique_regeo.copy()

In [361]:
testdf['polyLen'] = testdf.apply(lambda row: geomLen(row.geometry), axis=1)

In [362]:
testdf['geomZscore'] = np.abs(stats.zscore(testdf['polyLen']))

In [348]:
testdf.loc[testdf['geomZscore']>3]

,APN,APN2,MH_Geocoded_X,MH_Geocoded_Y,MHPID,MH_NAME,MH_ADDRESS,MH_CITY,MH_STATE,MH_ZIP,MH_STATUS,MH_COUNTY_FIPS,MH_LATITUDE,MH_LONGITUDE,geometry,VERSION,polyLen,geomZscore
120,183916400905,None,-105.682575,39.76097,12153.0,ROCKY MEADOWS TRAILER PARK,PO BOX 445,EMPIRE,CO,80438,OPEN,08019,39.761342,-105.6854,"POLYGON ((-105.78425 39.77697, -105.77240 39.7...",regeocoded,1665,15.318169


In [363]:
testdf.drop(testdf[testdf.geomZscore >3].index, inplace=True)

In [367]:
testdf.reset_index(inplace=True)

In [369]:
testdf.drop(['index'],axis=1, inplace=True)

In [ ]:
testdf

Albers versions of MHP datasets:

In [438]:
mhps_regeo = gpd.read_file(os.path.join(co_22_dir,'08_MHPS.gpkg'),layer='08_MHPs_Prepped')
mhps_orig = gpd.read_file(os.path.join(co_22_dir,'08_MHPS_OG.gpkg'),layer='08_MHPs_Prepped_OG')

In [439]:
mhps_regeo.to_crs(crs='ESRI:102003', inplace=True)
mhps_original.to_crs(crs='ESRI:102003', inplace=True)
mhps_regeo.to_file(os.path.join(co_22_dir,'08_MHPS_Albers.gpkg'),layer='08_MHPs_regeo_albers')
mhps_original.to_file(os.path.join(co_22_dir,'08_MHPS_Albers.gpkg'),layer='08_MHPs_original_albers')